Factorization Machines (https://www.csie.ntu.edu.tw/~b97053/paper/Rendle2010FM.pdf)
- a generic algorithm for classification, regression and ranking
- a generalization of linear regression and matrix factorization model
- similar to SVM with a polynomial kernel
- models n-way (typically 2-way) interaction between features (embeddings)
- compute 2-way feature interaction as the dot product of the latent features
- author proposes a neat trick to reduce polynomial complexity to linear time

the model (2-way interaction): 

$\hat{y}(x) = \mathbf{w}_0 + \sum_{i=1}^d \mathbf{w}_i x_i + \sum_{i=1}^d\sum_{j=i+1}^d \langle\mathbf{v}_i, \mathbf{v}_j\rangle x_i x_j$

- $w_0$ - global bias
- $w_i$ - weight of i-th variable
- $V, v_i, v_j$ - feature embeddings
- $\langle\mathbf{v}_i, \mathbf{v}_j\rangle$ - dot product of embeddings -> weight of the interaction

As we can see, the first part of the equation is just a linear regression

derivation of pairwise interaction

\begin{split}\begin{aligned}
&\sum_{i=1}^d \sum_{j=i+1}^d \langle\mathbf{v}_i, \mathbf{v}_j\rangle x_i x_j \\
 &= \frac{1}{2} \sum_{i=1}^d \sum_{j=1}^d\langle\mathbf{v}_i, \mathbf{v}_j\rangle x_i x_j - \frac{1}{2}\sum_{i=1}^d \langle\mathbf{v}_i, \mathbf{v}_i\rangle x_i x_i \\
 &= \frac{1}{2} \big (\sum_{i=1}^d \sum_{j=1}^d \sum_{l=1}^k\mathbf{v}_{i, l} \mathbf{v}_{j, l} x_i x_j - \sum_{i=1}^d \sum_{l=1}^k \mathbf{v}_{i, l} \mathbf{v}_{i, l} x_i x_i \big)\\
 &=  \frac{1}{2} \sum_{l=1}^k \big ((\sum_{i=1}^d \mathbf{v}_{i, l} x_i) (\sum_{j=1}^d \mathbf{v}_{j, l}x_j) - \sum_{i=1}^d \mathbf{v}_{i, l}^2 x_i^2 \big ) \\
 &= \frac{1}{2} \sum_{l=1}^k \big ((\sum_{i=1}^d \mathbf{v}_{i, l} x_i)^2 - \sum_{i=1}^d \mathbf{v}_{i, l}^2 x_i^2)
 \end{aligned}\end{split}

 - $k$ - embedding dimension
 - $d$ - number of features

An illustration of the shape & format of the data, where each feature will be embedded into a latent space

<img src="illustration.png" style="width:600px;height:300px;">


In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils import data
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
from sklearn.metrics import mean_squared_error, mean_absolute_error

Factorization Machines can be very easily implemented in Pytorch with just a few lines of code!

In [1]:
import torch
import torch.nn as nn
class FM(nn.Module):
    def __init__(self, n_features, embed_dim):
        super().__init__()
        self.feature_embedding = nn.Embedding(n_features, embed_dim) # n_dim feature embeddings for 2-way interaction
        self.fc = nn.Embedding(n_features, 1) # 1-d embedding equivalent to the weights for each feature in linear regression
        self.bias = nn.Parameter(torch.zeros((1,))) # global bias term
        
        
    def forward(self, x):
        first_order = self.fc(x).sum(dim=1) + self.bias
        second_order = self.factorization_machine(self.feature_embedding(x))
        res = first_order + second_order
        return res
    
    
    def factorization_machine(self, x_embed):
        """compute the 2-way interaction term
        """
        sq_sum = x_embed.sum(dim=1)**2 # (x_size, n_dim)
        sum_sq = (x_embed**2).sum(dim=1) # (x_size, n_dim)
        return 0.5 * (sq_sum - sum_sq).sum(dim=1, keepdim=True) # (x_size, 1)
    
    def predict(self, x):
        self.eval()
        with torch.no_grad():
            return self.forward(x)
    
        

The `factorization_machine()` function implements the equation:

$\frac{1}{2} \sum_{l=1}^k \big ((\sum_{i=1}^d \mathbf{v}_{i, l} x_i)^2 - \sum_{i=1}^d \mathbf{v}_{i, l}^2 x_i^2)$

There are two parts in the equation:

$(\mathbf{v}_{i, l} x_i)^2$     and     $(\mathbf{v}_{i, l}^2 x_i^2)$

Since each $x_i$ corresponds to one $v_i$, we can just treat $\mathbf{v}_{i, l} x_i$ as one latent embedding vector, which will be estimated during training, instead of estimating $V$ separately.

In [3]:
# train loop
def train(model, dataloader, epochs=20, lr=0.001):
    device = (
        torch.device("cuda:0") if torch.cuda.is_available(
        ) else torch.device("cpu")
    )
    model.to(device)
    model.train()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()
    training_history = []
    for epoch in range(epochs):
        epoch_loss = 0
        for x, y in dataloader:
            y_pred = model.forward(x)
            loss = criterion(y_pred, y)
            epoch_loss += loss
            model.zero_grad()
            loss.backward()
            optimizer.step()
        epoch_loss /= len(dataloader)
        training_history.append(epoch_loss)
        if epoch%10 == 0:
            print(f"Epoch {epoch}: {epoch_loss:.4f}")
    return model, training_history


# Data Preparation
- X is an array of feature indices (n_samples, n_attributes), where each feature index will be mapped to a latent factor
  - [user, item, user_features, item_features]
- y is just a (n_sample, 1) array of the ground truth

In [4]:
import sys
sys.path.append('..')
import utils

In [5]:
movie, rating = utils.load_movielens()

Encode movie features

In [6]:
# construct an array of feature indices for each item
all_movie_features = set([f for feat in movie['features'] for f in feat])
feature_to_id = {f:ix for ix, f in enumerate(all_movie_features)}
movie_feat_id = movie['features'].apply(lambda x: [feature_to_id[f]+1 for f in x]).to_list() # +1 since 0 is the null feature

# since items have variable # of features, pad feature sequence with 0 
features_enc = pad_sequence([torch.tensor(i) for i in movie_feat_id], batch_first=True, padding_value=0)

Combine userId, movieId and features to get our `x`
- [userId, movieId, feature-1, feature-n]

In [7]:
n_user = rating['userId'].nunique()
n_movies = movie['movieId'].nunique()

In [8]:
# join userid & movieid, and features
x = rating[['userId', 'movieId']].to_numpy()
features = features_enc[rating['movieId']]
x = np.hstack((x, features))
x

array([[   0,   30,    2, ...,    0,    0,    0],
       [   0,  833,   24, ...,    0,    0,    0],
       [   0,  859,   10, ...,    0,    0,    0],
       ...,
       [ 670, 4603,    3, ...,    0,    0,    0],
       [ 670, 4616,    2, ...,    0,    0,    0],
       [ 670, 4703,    2, ...,    0,    0,    0]])

In [9]:
# calculate offset, avoid duplicated feature indices
x[:, 1] += n_user
x[:, 2:] += n_user+n_movies
x

array([[   0,  701, 9798, ..., 9796, 9796, 9796],
       [   0, 1504, 9820, ..., 9796, 9796, 9796],
       [   0, 1530, 9806, ..., 9796, 9796, 9796],
       ...,
       [ 670, 5274, 9799, ..., 9796, 9796, 9796],
       [ 670, 5287, 9798, ..., 9796, 9796, 9796],
       [ 670, 5374, 9798, ..., 9796, 9796, 9796]])

In [10]:
y = rating['rating'].to_numpy().reshape(-1, 1)

Create train & test set
- train: first n-1 ratings per user (n is the number of ratings of the user)
- test: last/most recent rating per user

In [11]:
test_ix = [i for i,v in enumerate(x[:-1, 0]) if v != x[i+1,0]] + [len(x)-1]
train_ix = [i for i in range(len(x)) if i not in test_ix]

In [12]:
x_train, x_test = x[train_ix], x[test_ix]
y_train, y_test = y[train_ix], y[test_ix]

In [13]:
dataset = data.TensorDataset(torch.from_numpy(x_train), torch.from_numpy(y_train).float())
train_dataloader = DataLoader(dataset, batch_size=128, shuffle=True)

# Train Model

In [15]:
features.shape[-1]

11

In [25]:
n_features = n_user + n_movies + len(all_movie_features) + 1 # +1 for the null feature
embed_dim = 20

model = FM(n_features=n_features, embed_dim=embed_dim)

In [26]:
model, history = train(model, train_dataloader, epochs=300, lr=0.001)

Epoch 0: 98063.4766
Epoch 10: 78.0661
Epoch 20: 7.3351
Epoch 30: 1.5365
Epoch 40: 0.7909
Epoch 50: 0.6228
Epoch 60: 0.5518
Epoch 70: 0.5039
Epoch 80: 0.4661
Epoch 90: 0.4333
Epoch 100: 0.4006
Epoch 110: 0.3723
Epoch 120: 0.3476
Epoch 130: 0.3234
Epoch 140: 0.3024
Epoch 150: 0.2840
Epoch 160: 0.2689
Epoch 170: 0.2552
Epoch 180: 0.2414
Epoch 190: 0.2301
Epoch 200: 0.2197
Epoch 210: 0.2112
Epoch 220: 0.2035
Epoch 230: 0.1972
Epoch 240: 0.1901
Epoch 250: 0.1849
Epoch 260: 0.1797
Epoch 270: 0.1748
Epoch 280: 0.1703
Epoch 290: 0.1674


In [27]:
y_pred = model.predict(torch.from_numpy(x_test))

In [28]:
mean_squared_error(y_pred, y_test, squared=False)


2.4924757863411653

In [29]:
mean_absolute_error(y_pred, y_test)


1.7903649881593162

: 